## 1.朋友圈问题
* N*N二维数组，认识=1，任何人和自己认识m[i][i]=1，互相认识m[i][j]=1则m[j][i]=1.
* 0认识2,0认识4，所以024是一个朋友圈（即使2不认识4）
* 链接：https://leetcode.com/problems/number-of-provinces/

In [4]:

def findCircleNum(isConnected):
    N = len(isConnected)
    unionFind = UnionFind(N)
    for i in range(N): #只遍历右上半区
        for j in range(i+1, N):
            if isConnected[i][j] == 1:
                unionFind.union(i,j)
    return unionFind.sets

class UnionFind:
    def __init__(self, N):
        #parent[i]=k： i的父亲是k
        self.parent = [-1] * N 
        #size[i]=k：  如果i是代表节点，size[i]才有意义，表示以i为代表节点的集合大小
        self.size = [1] * N    
        self.help = [-1] * N
        #一共多少集合
        self.sets = N
        for i in range(N):
            self.parent[i] = i
    #从i开始一直往上，往上到不能再往上，代表节点返回。此过程完成路径压缩
    def find(self, i):
        hi = 0
        while i != self.parent[i]:
            self.help[hi] = i #help数组记录沿途经过的i
            i = self.parent[i] #往上查
            hi += 1
        #跳出while时刻，i为代表节点
        hi -= 1
        while hi >= 0:
            self.parent[self.help[hi]] = i #路径压缩
            hi -= 1
        return i
    
    def union(self, i, j):
        f1 = self.find(i)
        f2 = self.find(j)
        if f1!=f2:
            if self.size[f1] >= self.size[f2]:
                self.size[f1] += self.size[f2]
                self.parent[f2] = f1
            else:
                self.size[f2] += self.size[f1]
                self.parent[f1] = f2
            self.sets -= 1
        

## 2.岛问题
* 给定一个二维数组matrix，相邻的1认为是一个岛，问有多少个岛
* 链接：https://leetcode.com/problems/number-of-islands/

In [6]:
#递归版本
#从i,j出发，将相邻1的所到之处"1"变成"2"
def numIslands(grid):
    islands = 0
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == "1":
                islands += 1
                infects(grid, i, j)
    return islands

def infects(board, i, j):
    if i < 0 or i == len(board) or j < 0 or j == len(board[0]) or board[i][j] != "1":
        return 
    board[i][j] = 2
    infects(board, i-1, j)
    infects(board, i+1, j)
    infects(board, i, j-1)
    infects(board, i, j+1)
    

In [7]:
#并查集版本
#从i,j出发，只查自己的左上，有1合并

In [22]:
class UnionFind2:
    def __init__(self, grid):
        self.N = len(grid)
        self.M = len(grid[0])
        self.parent = [-1]*(self.N*self.M) 
        self.size = [1]*(self.N*self.M)
        self.help = [-1]*(self.N*self.M) 
        self.sets = 0
        for i in range(self.N):
            for j in range(self.M):
                if grid[i][j] == "1":
                    index = self.__getIndex__(i,j)
                    self.parent[index] = index
                    self.sets +=1
    def __getIndex__(self, x, y):
        return int(x * self.M + y)
    def find(self, x, y):
        hi = 0
        index = self.__getIndex__(x,y)
        while index != self.parent[index]:
            self.help[hi] = index
            hi += 1
            index = self.parent[index]
        #index == self.parent[index]代表节点
        hi -= 1
        for i in range(0, hi):
            self.parent[self.help[i]] = index
        return index
    def union(self, x1, y1, x2, y2):
        f1 = self.find(x1, y1)
        f2 = self.find(x2, y2)
        if f1 != f2:
            if self.size[f1] >= self.size[f2]:
                self.parent[f2] = f1
                self.size[f1] = self.size[f1] + self.size[f2]
            else:
                self.parent[f1] = f2
                self.size[f2] = self.size[f1] + self.size[f2]
            self.sets -= 1
        return
    
def numIslands2(grid):
    unf = UnionFind2(grid)
    N = len(grid)
    M = len(grid[0])
    for j in range(1, N):
        if grid[0][j] == "1" and grid[0][j-1] == "1":
            unf.union(0, j-1, 0, j)
    for i in range(1, M):
        if grid[i][0] == "1" and grid[i-1][0] == "1":
            unf.union(i-1, 0, i, 0)
    for i in range(1, N):
        for j in range(1, M):
            if grid[i][j] == "1":
                if grid[i][j-1] == "1":
                    unf.union(i, j-1, i, j)
                if grid[i-1][j] == "1":
                    unf.union(i-1, j, i, j)
    return unf.sets
                
        

## 3.岛位置2
* 给定N,M大小的范围，初始都是0.每次给一个位置，返回岛数。
* 输入为 m,n,positions. m、n为正整数，position为[(x,y), (x2,y2)...]
* https://leetcode.com/problems/number-of-islands-ii/ （有锁）

In [21]:
#涉及动态感染(合并)过程
#用size==0表示没被初始化，当没被初始化，此时空降了一个点，再初始化这个点的状态(parent,size,sets).
#初始化完成后，尝试上下左右union，形成当前时刻的岛状态。

class UnionFind3:
    def __init__(self, N, M):
        self.N = N
        self.M = M
        self.parent = [-1]*(N*M) 
        self.size = [0]*(N*M)
        self.help = [-1]*(N*M) 
        self.sets = 0
        #每次的parent初始化通过connect完成
    def __getIndex__(self, x, y):
        return int(x * self.M + y)
    def find(self, x, y):
        hi = 0
        index = self.__getIndex__(x,y)
        while index != self.parent[index]:
            self.help[hi] = index
            hi += 1
            index = self.parent[index]
        #index == self.parent[index]代表节点
        hi -= 1
        for i in range(0, hi):
            self.parent[self.help[i]] = index
        return index
    
    def union(self, x1, y1, x2, y2):
        if x1 < 0 or x1 == self.N or x2 < 0 or x2 == self.N or  \
            y1 < 0 or y1 == self.M or y2 < 0 or y2 == self.M:
                return
        i1 = self.__getIndex__(x1,y1)
        i2 = self.__getIndex__(x2,y2)
        #如果有一个还未初始化，直接跳过
        if size[i1] == 0 or size[i2] == 0:
            return 
        f1 = self.find(x1, y1)
        f2 = self.find(x2, y2)

        if f1 != f2:
            if self.size[f1] >= self.size[f2]:
                self.parent[f2] = f1
                self.size[f1] = self.size[f1] + self.size[f2]
            else:
                self.parent[f1] = f2
                self.size[f2] = self.size[f1] + self.size[f2]
            self.sets -= 1
        return
    
    def connect(self, x, y):
        index = self.__getIndex__(x,y)
        #如果重复空降，直接跳过
        if self.size[index] == 0:
            self.parent[index] = index
            self.size[index] = 1
            self.sets += 1
            self.union(x-1, y, x, y)
            self.union(x+1, y, x, y)
            self.union(x, y-1, x, y)
            self.union(x, y+1, x, y)
        return self.sets
def numIslands3(m, n, positions):
    uf = UnionFind3(m, n)
    ans = list()
    for position in positions:
        ans.append(uf.connect(position[0],position[1]))
    return ans
#上面的初始化是O(N*M)，每次connect的是O(1)。如果N*M很大，那么可以用hashmap

## 4.岛问题(扩展)
如果matrix很大，要求设计并行的方案

In [23]:
'''
思路：
1. 并行 分别感染两边的岛，并记录每个岛的代表节点，返回各自岛数，求和整体岛数
2. 考虑岛边界(上下左右四个)左右两侧的节点，如果两侧都被感染成了“2”，且两侧的节点代表节点不是同一个，合并(并查集),整体岛数-1
'''

'\n思路：\n1. 并行 分别感染两边的岛，并记录每个岛的代表节点，返回各自岛数，求和整体岛数\n2. 考虑岛边界左右两侧的节点，如果两侧都被感染成了“2”，且两侧的节点代表节点不是同一个，合并(并查集),整体岛数-1\n'